In [1]:
### We create a bunch of helpful functions throughout the course.
### Storing them here so they're easily accessible.

import tensorflow as tf

# Create a function to import an image and resize it to be able to be used with our model
def load_and_prep_image(filename, img_shape=224, scale=True):
  """
  Reads in an image from filename, turns it into a tensor and reshapes into
  (224, 224, 3).

  Parameters
  ----------
  filename (str): string filename of target image
  img_shape (int): size to resize target image to, default 224
  scale (bool): whether to scale pixel values to range(0, 1), default True
  """
  # Read in the image
  img = tf.io.read_file(filename)
  # Decode it into a tensor
  img = tf.image.decode_jpeg(img)
  # Resize the image
  img = tf.image.resize(img, [img_shape, img_shape])
  if scale:
    # Rescale the image (get all values between 0 and 1)
    return img/255.
  else:
    return img

# Note: The following confusion matrix code is a remix of Scikit-Learn's 
# plot_confusion_matrix function - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html
import itertools
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix

# Our function needs a different name to sklearn's plot_confusion_matrix
def make_confusion_matrix(y_true, y_pred, classes=None, figsize=(10, 10), text_size=15, norm=False, savefig=False): 
  """Makes a labelled confusion matrix comparing predictions and ground truth labels.

  If classes is passed, confusion matrix will be labelled, if not, integer class values
  will be used.

  Args:
    y_true: Array of truth labels (must be same shape as y_pred).
    y_pred: Array of predicted labels (must be same shape as y_true).
    classes: Array of class labels (e.g. string form). If `None`, integer labels are used.
    figsize: Size of output figure (default=(10, 10)).
    text_size: Size of output figure text (default=15).
    norm: normalize values or not (default=False).
    savefig: save confusion matrix to file (default=False).
  
  Returns:
    A labelled confusion matrix plot comparing y_true and y_pred.

  Example usage:
    make_confusion_matrix(y_true=test_labels, # ground truth test labels
                          y_pred=y_preds, # predicted labels
                          classes=class_names, # array of class label names
                          figsize=(15, 15),
                          text_size=10)
  """  
  # Create the confustion matrix
  cm = confusion_matrix(y_true, y_pred)
  cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis] # normalize it
  n_classes = cm.shape[0] # find the number of classes we're dealing with

  # Plot the figure and make it pretty
  fig, ax = plt.subplots(figsize=figsize)
  cax = ax.matshow(cm, cmap=plt.cm.Blues) # colors will represent how 'correct' a class is, darker == better
  fig.colorbar(cax)

  # Are there a list of classes?
  if classes:
    labels = classes
  else:
    labels = np.arange(cm.shape[0])
  
  # Label the axes
  ax.set(title="Confusion Matrix",
         xlabel="Predicted label",
         ylabel="True label",
         xticks=np.arange(n_classes), # create enough axis slots for each class
         yticks=np.arange(n_classes), 
         xticklabels=labels, # axes will labeled with class names (if they exist) or ints
         yticklabels=labels)
  
  # Make x-axis labels appear on bottom
  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  # Set the threshold for different colors
  threshold = (cm.max() + cm.min()) / 2.

  # Plot the text on each cell
  for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    if norm:
      plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j]*100:.1f}%)",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)
    else:
      plt.text(j, i, f"{cm[i, j]}",
              horizontalalignment="center",
              color="white" if cm[i, j] > threshold else "black",
              size=text_size)

  # Save the figure to the current working directory
  if savefig:
    fig.savefig("confusion_matrix.png")
  
# Make a function to predict on images and plot them (works with multi-class)
def pred_and_plot(model, filename, class_names):
  """
  Imports an image located at filename, makes a prediction on it with
  a trained model and plots the image with the predicted class as the title.
  """
  # Import the target image and preprocess it
  img = load_and_prep_image(filename)

  # Make a prediction
  pred = model.predict(tf.expand_dims(img, axis=0))

  # Get the predicted class
  if len(pred[0]) > 1: # check for multi-class
    pred_class = class_names[pred.argmax()] # if more than one output, take the max
  else:
    pred_class = class_names[int(tf.round(pred)[0][0])] # if only one output, round

  # Plot the image and predicted class
  plt.imshow(img)
  plt.title(f"Prediction: {pred_class}")
  plt.axis(False);
  
import datetime

def create_tensorboard_callback(dir_name, experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.

  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"

  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

# Plot the validation and training data separately
import matplotlib.pyplot as plt

def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.

  Args:
    history: TensorFlow model History object (see: https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/History)
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

def compare_historys(original_history, new_history, initial_epochs=5):
    """
    Compares two TensorFlow model History objects.
    
    Args:
      original_history: History object from original model (before new_history)
      new_history: History object from continued model training (after original_history)
      initial_epochs: Number of epochs in original_history (new_history plot starts from here) 
    """
    
    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    # Make plots
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()
  
# Create function to unzip a zipfile into current working directory 
# (since we're going to be downloading and unzipping a few files)
import zipfile

def unzip_data(filename):
  """
  Unzips filename into the current working directory.

  Args:
    filename (str): a filepath to a target zip folder to be unzipped.
  """
  zip_ref = zipfile.ZipFile(filename, "r")
  zip_ref.extractall()
  zip_ref.close()

# Walk through an image classification directory and find out how many files (images)
# are in each subdirectory.
import os

def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.

  Args:
    dir_path (str): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")
    
# Function to evaluate: accuracy, precision, recall, f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calculates model accuracy, precision, recall and f1 score of a binary classification model.

  Args:
      y_true: true labels in the form of a 1D array
      y_pred: predicted labels in the form of a 1D array

  Returns a dictionary of accuracy, precision, recall, f1-score.
  """
  # Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  # Calculate model precision, recall and f1 score using "weighted average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results


C:\Users\asus\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
path_to_data ="C:\\Users\\asus\\\Desktop\\Udemy NLP\\"

In [3]:
os.listdir(path_to_data)

['test.csv', 'train.csv']

# Visualizing the data 

In [4]:
import pandas as pd
import numpy as np

In [5]:
data_train = pd.read_csv(path_to_data+"train.csv")
data_test = pd.read_csv(path_to_data+"test.csv")

In [6]:
data_train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [7]:
data_test

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


# EDA

In [8]:
data_train[data_train['target']==0]['text']
# these are the types of tweets which have no disaster associated with them 

15                                         What's up man?
16                                          I love fruits
17                                       Summer is lovely
18                                      My car is so fast
19                           What a goooooooaaaaaal!!!!!!
                              ...                        
7581    @engineshed Great atmosphere at the British Li...
7582    Cramer: Iger's 3 words that wrecked Disney's s...
7584    These boxes are ready to explode! Exploding Ki...
7587                                   Sirens everywhere!
7593    I just heard a really loud bang and everyone i...
Name: text, Length: 4342, dtype: object

In [9]:
# now look at the disaster tweets 

In [10]:
data_train.target.value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [11]:
data_shuffled = data_train.sample(frac=1,random_state=42)

In [12]:
data_shuffled

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0
...,...,...,...,...,...
5226,7470,obliteration,Merica!,@Eganator2000 There aren't many Obliteration s...,0
5390,7691,panic,NaN,just had a panic attack bc I don't have enough...,0
860,1242,blood,NaN,Omron HEM-712C Automatic Blood Pressure Monito...,0
7603,10862,NaN,NaN,Officials say a quarantine is in place at an A...,1


In [13]:
# visualizng random data
import random
random_index = random.randint(0,len(data_train)-5)
for row in data_shuffled[['text','target']][random_index:random_index+5].itertuples():
    _,text,target = row
    print("(actual disaster)\n target = 1 " if target>0 else "(not disaster)\n target = 0")
    print("{}".format(text))
    print("----")
    

(not disaster)
 target = 0
Retweeted Sarah Silverman (@SarahKSilverman):

Soundtrack of my walk starts w Tracey Ullman They Don't Know.  perfect
----
(actual disaster)
 target = 1 
I liked a @YouTube video http://t.co/bGAJ2oAX1p Huge Building Explosion at 2500fps - The Slow Mo Guys
----
(not disaster)
 target = 0
If I survive I'll see you tomorrow
----
(not disaster)
 target = 0
You wanna ruin a relationship? Just ask 'what are your intentions' and you'll do it
----
(not disaster)
 target = 0
Blood Group  A +ve is associated with Gastric Carcinoma says text book...Another fragile gene in my body....
----


# Splitting the data into train and validation set



In [14]:
from sklearn.model_selection import train_test_split

In [15]:
train_sentences ,val_sentences , train_target, val_target = train_test_split(data_shuffled['text'].to_numpy(),
                                                                             data_shuffled['target'].to_numpy(),
                                                                             test_size = 0.1,
                                                                             random_state = 42
                                                                            )

In [16]:
train_sentences

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       ...,
       'Near them on the sand half sunk a shattered visage lies... http://t.co/0kCCG1BT06',
       "kesabaran membuahkan hasil indah pada saat tepat! life isn't about waiting for the storm to pass it's about learning to dance in the rain.",
       "@ScottDPierce @billharris_tv @HarrisGle @Beezersun I'm forfeiting this years fantasy football pool out of fear I may win n get my ass kicked"],
      dtype=object)

In [17]:
train_target

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

# Text to numbers
there are a few ways to do it -:
1. tokenization -: assign a number to each token in the sentence . i.e assigning a number to a word , or a letter or other. 
2. Embedding -: creating a feature matrix for each Token.

In [18]:
# setting up text vectorization 
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [19]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Note: in TensorFlow 2.6+, you no longer need "layers.experimental.preprocessing"
# you can use: "tf.keras.layers.TextVectorization", see https://github.com/tensorflow/tensorflow/releases/tag/v2.6.0 for more

# Use the default TextVectorization variables
text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (all of the different words in your text)
                                    standardize="lower_and_strip_punctuation", # how to process text
                                    split="whitespace", # how to split tokens
                                    ngrams=None, # create groups of n-words?
                                    output_mode="int", # how to map tokens to numbers
                                    output_sequence_length=None) # how long should the output sequence of tokens be?
                                    # pad_to_max_tokens=True) # Not valid if using max_tokens=None

In [20]:
len(train_sentences[0].split())

7

In [21]:
# how many token per sentence average?
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [22]:
# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

### Mapping the vectorizer to the training sentence

In [23]:
# Fit the text vectorizer to the training text
text_vectorizer.adapt(train_sentences)

In [24]:
sample_sentence = 'hi this is Reyan!!!'
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1663,   19,    9,    1,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int64)>

In [25]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n{random_sentence}\
      \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
incident with injury:I-495  inner loop Exit 31 - MD 97/Georgia Ave Silver Spring      

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[1656,   14, 2423, 1655, 2062, 1681, 1563, 1814, 2578, 1094, 1112,
        1110,    0,    0,    0]], dtype=int64)>

In [26]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5] # most common tokens (notice the [UNK] token for "unknown" words)
bottom_5_words = words_in_vocab[-5:] # least common tokens
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"Top 5 most common words: {top_5_words}") 
print(f"Bottom 5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
Top 5 most common words: ['', '[UNK]', 'the', 'a', 'in']
Bottom 5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


# Embedding 

In [27]:
from tensorflow.keras.layers import Embedding

In [28]:
tf.random.set_seed(42)
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding_1") 

embedding

In [29]:
embed_sent = embedding(text_vectorizer([i for i in train_sentences]))


In [30]:
embed_sent[1].shape

TensorShape([15, 128])

In [31]:
embed_sent[1][0].shape

TensorShape([128])

In [32]:
random_sentence = random.choice(train_sentences)
print(f"random Sentence -: \n {random_sentence}\n\n")
print(f"embeded sentence -: \n {embedding(text_vectorizer([random_sentence]))}")

random Sentence -: 
 Rip ?? Blood !


embeded sentence -: 
 [[[-0.03096329  0.02698782  0.04485491 ... -0.02351606  0.00890239
   -0.03530603]
  [-0.04425305  0.04668229  0.00983556 ...  0.04071194 -0.01522639
   -0.01229566]
  [ 0.01445952  0.04389951 -0.00408784 ... -0.00873065 -0.0236426
    0.03486079]
  ...
  [ 0.01445952  0.04389951 -0.00408784 ... -0.00873065 -0.0236426
    0.03486079]
  [ 0.01445952  0.04389951 -0.00408784 ... -0.00873065 -0.0236426
    0.03486079]
  [ 0.01445952  0.04389951 -0.00408784 ... -0.00873065 -0.0236426
    0.03486079]]]


### Experimenting with different models

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

In [34]:
model_0 = Pipeline([
    ('tfIdf',TfidfVectorizer()),
    ('clf',MultinomialNB())
])

In [35]:
model_0.fit(train_sentences,train_target)

Pipeline(steps=[('tfIdf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [36]:
baseline_pred = model_0.predict(val_sentences)

In [37]:
model_0.score(val_sentences,val_target)

0.7926509186351706

In [38]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

In [39]:
def give_model_score(y_true,y_pred):
    accuracy = accuracy_score(y_true,y_pred)
    precision,recall,fscore,_  = precision_recall_fscore_support(y_true,y_pred,average ='weighted')
    result = {'accuracy':np.round(accuracy,2)*100,'precision':np.round(precision,2),'recall':np.round(recall,2),'fscore':np.round(fscore,2)}
    return result

In [40]:
give_model_score(baseline_pred,val_target)
# val_target
# baseline_pred

{'accuracy': 79.0, 'precision': 0.83, 'recall': 0.79, 'fscore': 0.8}

##  Model 1. Dense Model

In [41]:
# building using the Functional API

from tensorflow.keras import layers
log_dir = "C:\\Users\\asus\\Desktop\\model callbacks"
tv =TextVectorization(max_tokens = 10000, standardize = 'lower_and_strip_punctuation',output_mode='int',pad_to_max_tokens=True,output_sequence_length=15)

In [42]:
# Build model with the Functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string") # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numerized numbers
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create the output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense") # construct the model

In [43]:
model_1 = tf.keras.Model(inputs,outputs)

In [44]:
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-train

In [45]:
# Compile model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [46]:
history_1 = model_1.fit(train_sentences,
            train_target,
            epochs =5,
            validation_data = (val_sentences,val_target),
            callbacks =create_tensorboard_callback(dir_name =log_dir,experiment_name='model_1'))

Saving TensorBoard log files to: C:\Users\asus\Desktop\model callbacks/model_1/20230503-190716
Epoch 1/5
215/215 [==============================] - 3s 13ms/step - loss: 0.6088 - accuracy: 0.6919 - val_loss: 0.5353 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4407 - accuracy: 0.8199 - val_loss: 0.4689 - val_accuracy: 0.7887
Epoch 3/5
215/215 [==============================] - 2s 12ms/step - loss: 0.3459 - accuracy: 0.8628 - val_loss: 0.4589 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2843 - accuracy: 0.8926 - val_loss: 0.4642 - val_accuracy: 0.7900
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.2374 - accuracy: 0.9123 - val_loss: 0.4769 - val_accuracy: 0.7874


In [47]:
pred =  model_1.predict(val_sentences)

24/24 [==============================] - 0s 940us/step


In [48]:
for i in range(len(pred)) :
    if(pred[i]>0.5):
        pred[i] = 1
    else :
        pred[i] =0 

In [49]:
give_model_score(val_target,pred)

{'accuracy': 79.0, 'precision': 0.79, 'recall': 0.79, 'fscore': 0.78}

In [50]:
model_results = calculate_results(val_target,pred)

In [51]:
model_results

{'accuracy': 78.74015748031496,
 'precision': 0.791897156051696,
 'recall': 0.7874015748031497,
 'f1': 0.7845557264291478}

In [52]:
baseline_result = calculate_results(val_target,baseline_pred)
baseline_result

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [53]:
for i in model_results:
    if(model_results[i]>baseline_result[i]):
        print('True')
    else :
        print('False')

False
False
False
False


In [54]:
np.array(list(model_results.values()))>np.array(list(baseline_result.values()))

array([False, False, False, False])

# Visualizing the models embedding learnt

In [55]:
model_1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
Non-train

In [56]:
embed_weights = model_1.get_layer('embedding_1').get_weights()[0]
print(embed_weights)
print(embed_weights.shape)

[[ 0.02609496  0.03379224 -0.0175851  ... -0.02017951 -0.0334495
   0.02453345]
 [-0.04135446  0.00246227 -0.03937959 ...  0.02725996  0.02878668
  -0.03615825]
 [-0.01908249 -0.04771005 -0.02814157 ... -0.06276737  0.02449374
   0.00107317]
 ...
 [-0.03849956  0.02226267 -0.03091577 ... -0.0250837  -0.00960133
   0.01363235]
 [ 0.02360207 -0.0052059  -0.08515263 ... -0.0147745  -0.07742629
  -0.0104909 ]
 [ 0.07534593 -0.0925798  -0.04149812 ... -0.09095236 -0.05529407
  -0.05488086]]
(10000, 128)


### to visualize what our model is seeing we need to use -> projector.tensorflow.org

In [57]:
import io

In [58]:
out_v = io.open('vector.tsv','w',encoding='utf-8')
out_m = io.open('metadata.tsv','w',encoding ='utf-8')
for index , word in enumerate(words_in_vocab):
    if index==0:
        continue
    vec = embed_weights[index]
    out_v.write('\t'.join([str(x) for x in vec])+'\n')
    out_m.write(word+'\n')
    
out_v.close()
out_m.close()

        

**Resources -:** 
1. Chris Olah LSTM's 
2. Almaarey Word2Vec 
3. MIT Video on RNN and NLP 

# Recurrent Neural Network

In [59]:
# lstm layer
from tensorflow.keras import layers
Input = tf.keras.layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(Input)
x = embedding(x)
x= tf.keras.layers.LSTM(64)(x)
output = tf.keras.layers.Dense(1,activation='sigmoid')(x)
model_2 = tf.keras.Model(Input,output,name='lstm_model')


In [60]:
model_2.compile(loss='binary_crossentropy',optimizer ='adam',metrics=['accuracy'])


In [61]:
model_2.fit(train_sentences,train_target,epochs=5,validation_data=(val_sentences,val_target))

Epoch 1/5
215/215 [==============================] - 6s 19ms/step - loss: 0.2186 - accuracy: 0.9218 - val_loss: 0.5398 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.1600 - accuracy: 0.9416 - val_loss: 0.6035 - val_accuracy: 0.7782
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.1264 - accuracy: 0.9526 - val_loss: 0.7457 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 4s 16ms/step - loss: 0.1062 - accuracy: 0.9603 - val_loss: 0.8330 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0832 - accuracy: 0.9656 - val_loss: 0.9789 - val_accuracy: 0.7835


In [62]:
train_target

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [63]:
model_2.summary()

Model: "lstm_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
______________________________________________

In [64]:
model_pred = model_2.predict(val_sentences)

24/24 [==============================] - 0s 4ms/step


In [65]:
model_pred[:10]

array([[4.1364930e-03],
       [7.0667553e-01],
       [9.9963975e-01],
       [2.7757620e-02],
       [1.4344305e-04],
       [9.9799907e-01],
       [6.9244409e-01],
       [9.9981844e-01],
       [9.9966013e-01],
       [5.3189486e-01]], dtype=float32)

In [66]:
model_pred.shape

(762, 1)

In [67]:
pred_label = [1 if i>0.5 else 0 for i in model_pred ]

In [68]:
pred_label[:10]

[0, 1, 1, 0, 0, 1, 1, 1, 1, 1]

In [69]:
pred_class = ['disaster' if i==1 else 'not a disaster' for i in pred_label]

In [70]:
pred_class

['not a disaster',
 'disaster',
 'disaster',
 'not a disaster',
 'not a disaster',
 'disaster',
 'disaster',
 'disaster',
 'disaster',
 'disaster',
 'not a disaster',
 'disaster',
 'not a disaster',
 'not a disaster',
 'not a disaster',
 'not a disaster',
 'not a disaster',
 'not a disaster',
 'not a disaster',
 'disaster',
 'disaster',
 'not a disaster',
 'disaster',
 'not a disaster',
 'disaster',
 'disaster',
 'not a disaster',
 'not a disaster',
 'not a disaster',
 'not a disaster',
 'disaster',
 'not a disaster',
 'not a disaster',
 'not a disaster',
 'disaster',
 'not a disaster',
 'disaster',
 'not a disaster',
 'not a disaster',
 'disaster',
 'not a disaster',
 'not a disaster',
 'not a disaster',
 'not a disaster',
 'disaster',
 'disaster',
 'disaster',
 'disaster',
 'not a disaster',
 'not a disaster',
 'not a disaster',
 'disaster',
 'not a disaster',
 'not a disaster',
 'disaster',
 'not a disaster',
 'not a disaster',
 'disaster',
 'not a disaster',
 'not a disaster',
 'no

In [71]:
ans = list(zip(val_sentences,pred_class))

In [72]:
ans

[('DFR EP016 Monthly Meltdown - On Dnbheaven 2015.08.06 http://t.co/EjKRf8N8A8 #Drum and Bass #heavy #nasty http://t.co/SPHWE6wFI5',
  'not a disaster'),
 ('FedEx no longer to transport bioterror germs in wake of anthrax lab mishaps http://t.co/qZQc8WWwcN via @usatoday',
  'disaster'),
 ('Gunmen kill four in El Salvador bus attack: Suspected Salvadoran gang members killed four people and wounded s... http://t.co/CNtwB6ScZj',
  'disaster'),
 ('@camilacabello97 Internally and externally screaming', 'not a disaster'),
 ('Radiation emergency #preparedness starts with knowing to: get inside stay inside and stay tuned http://t.co/RFFPqBAz2F via @CDCgov',
  'not a disaster'),
 ('Investigators rule catastrophic structural failure resulted in 2014 Virg.. Related Articles: http://t.co/Cy1LFeNyV8',
  'disaster'),
 ('How the West was burned: Thousands of wildfires ablaze in #California alone http://t.co/iCSjGZ9tE1 #climate #energy http://t.co/9FxmN0l0Bd',
  'disaster'),
 ("Map: Typhoon Soudelor's 

# GRU Layer -  Gated Recurrent Unit
reference -: Towardsdatascience



from tensorflow.keras import layers
Input = tf.keras.layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(Input)
x = embedding(x)
x= tf.keras.layers.LSTM(64)(x)
output = tf.keras.layers.Dense(1,activation='sigmoid')(x)
model_2 = tf.keras.Model(Input,output,name='lstm_model')


In [73]:
# lstm layer
from tensorflow.keras import layers
Input = tf.keras.layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(Input)
x = embedding(x)
x= tf.keras.layers.GRU(64)(x)
output = tf.keras.layers.Dense(1,activation='sigmoid')(x)
model_3 = tf.keras.Model(Input,output,name='GRU')


In [74]:
model_3.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 gru (GRU)                   (None, 64)                37248     
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________________

In [75]:
model_3.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [76]:
model_3.fit(train_sentences,train_target,epochs=5,validation_data=(val_sentences,val_target),callbacks=create_tensorboard_callback(log_dir,'gru_model'))

Saving TensorBoard log files to: C:\Users\asus\Desktop\model callbacks/gru_model/20230503-190752
Epoch 1/5
215/215 [==============================] - 6s 18ms/step - loss: 0.1561 - accuracy: 0.9397 - val_loss: 0.6550 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 4s 18ms/step - loss: 0.0852 - accuracy: 0.9688 - val_loss: 0.8062 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0710 - accuracy: 0.9726 - val_loss: 1.0402 - val_accuracy: 0.7782
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0658 - accuracy: 0.9743 - val_loss: 1.0676 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0542 - accuracy: 0.9765 - val_loss: 1.1428 - val_accuracy: 0.7769


In [77]:
preds3 = model_3.predict(val_sentences)

24/24 [==============================] - 0s 3ms/step


In [78]:
preds3 = [1 if i>0.5 else 0 for i in preds3]

# Bidirectional RNN
**Normal RNN goes from left to right but directional RNN goes from right to left and left to right.**

In [79]:
from tensorflow.keras.layers import Bidirectional

In [80]:
from tensorflow.keras import layers
inputs = tf.keras.layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(x)
x = Bidirectional(tf.keras.layers.GRU(64))(x)
output = tf.keras.layers.Dense(1,activation='sigmoid')(x)
model_4 = tf.keras.Model(inputs,output)

In [81]:
model_4.compile(loss='binary_crossentropy',optimizer ='adam',metrics =['accuracy'])
model_4.fit(train_sentences,train_target,epochs =5,validation_data=(val_sentences,val_target),callbacks=create_tensorboard_callback(log_dir,'bidirectional_model'))


Saving TensorBoard log files to: C:\Users\asus\Desktop\model callbacks/bidirectional_model/20230503-190814
Epoch 1/5
215/215 [==============================] - 14s 35ms/step - loss: 0.1007 - accuracy: 0.9682 - val_loss: 1.0097 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0547 - accuracy: 0.9768 - val_loss: 1.2197 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0483 - accuracy: 0.9784 - val_loss: 1.2550 - val_accuracy: 0.7730
Epoch 4/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0435 - accuracy: 0.9788 - val_loss: 1.3275 - val_accuracy: 0.7730
Epoch 5/5
215/215 [==============================] - 6s 29ms/step - loss: 0.0392 - accuracy: 0.9820 - val_loss: 1.5720 - val_accuracy: 0.7690


In [82]:
model_4.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                           

In [83]:
# t2d = np.array([[1,2,32,1,23,4,4,2,1]])
# sq = tf.squeeze(t2d).numpy()
# sq
preds4 = model_4.predict(val_sentences)
preds4 = [1 if i>0.5 else 0 for i in preds4]

24/24 [==============================] - 1s 7ms/step


In [84]:
calculate_results(val_target,preds4)
# worst results  by adding bidirectionality

{'accuracy': 76.9028871391076,
 'precision': 0.7701356893081864,
 'recall': 0.7690288713910761,
 'f1': 0.7671875324347506}

# 1D Conv Layers

In [85]:
from tensorflow.keras.layers import Conv1D

In [86]:
import tensorflow.keras.layers

In [106]:
input_layer = tf.keras.layers.Input(shape =(1,),dtype="string")
embedding_test = embedding(text_vectorizer(input_layer))
conv1D = Conv1D(filters = 64,
                kernel_size = 5,
                strides = 1,# default 
                activation ="relu",
                padding ="same"
               )

conv1D_output = conv1D(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv1D_output)
output = layers.Dense(1,activation =  "sigmoid")(max_pool_output)
model_5 = tf.keras.Model(input_layer,output)

In [107]:
embedding_test.shape,conv1D_output.shape,max_pool_output.shape 

(TensorShape([None, 15, 128]),
 TensorShape([None, 15, 64]),
 TensorShape([None, 64]))

In [108]:
model_5.compile(loss= "binary_crossentropy",optimizer ="adam",metrics =['accuracy'])

In [109]:
hist5 = model_5.fit(train_sentences,train_target,validation_data=(val_sentences,val_target),epochs = 5)

Epoch 1/5
215/215 [==============================] - 4s 13ms/step - loss: 0.0902 - accuracy: 0.9720 - val_loss: 0.9701 - val_accuracy: 0.7730
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0520 - accuracy: 0.9790 - val_loss: 1.1225 - val_accuracy: 0.7625
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0478 - accuracy: 0.9794 - val_loss: 1.1638 - val_accuracy: 0.7612
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0457 - accuracy: 0.9788 - val_loss: 1.2154 - val_accuracy: 0.7598
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0431 - accuracy: 0.9806 - val_loss: 1.2563 - val_accuracy: 0.7677


In [113]:
preds5 = model_5.predict(val_sentences)
preds5 = [1 if i>0.5 else 0 for i in preds5]

24/24 [==============================] - 0s 2ms/step


In [115]:
calculate_results(val_target,preds5)

{'accuracy': 76.77165354330708,
 'precision': 0.7686883789989795,
 'recall': 0.7677165354330708,
 'f1': 0.7659262393850468}

In [117]:
model_5.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 128)           1280000   
                                                                 
 conv1d_10 (Conv1D)          (None, 15, 64)            41024     
                                                                 
 global_max_pooling1d_8 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_7 (Dense)             (None, 1)                 65  

In [129]:
train_sentences.shape[0]/5

1370.2

# Transfer Learning

In [135]:
import tensorflow_hub as hub

In [137]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [164]:
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                             input_shape= [],
                             trainable = False,
                             dtype="string"
                            )

In [165]:
model_6= tf.keras.Sequential([
    sentence_encoder_layer,
    tf.keras.layers.Dense(1,"sigmoid")
])

In [166]:
model_6.compile(loss= "binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [174]:
model_6.fit(train_sentences,train_target,validation_data=(val_sentences,val_target),epochs =10)

Epoch 1/10
215/215 [==============================] - 2s 10ms/step - loss: 0.5383 - accuracy: 0.7939 - val_loss: 0.5309 - val_accuracy: 0.7808
Epoch 2/10
215/215 [==============================] - 2s 9ms/step - loss: 0.5098 - accuracy: 0.7996 - val_loss: 0.5098 - val_accuracy: 0.7861
Epoch 3/10
215/215 [==============================] - 2s 8ms/step - loss: 0.4897 - accuracy: 0.8003 - val_loss: 0.4954 - val_accuracy: 0.7848
Epoch 4/10
215/215 [==============================] - 2s 9ms/step - loss: 0.4750 - accuracy: 0.8013 - val_loss: 0.4850 - val_accuracy: 0.7887
Epoch 5/10
215/215 [==============================] - 2s 9ms/step - loss: 0.4639 - accuracy: 0.8046 - val_loss: 0.4775 - val_accuracy: 0.7913
Epoch 6/10
215/215 [==============================] - 2s 9ms/step - loss: 0.4552 - accuracy: 0.8054 - val_loss: 0.4714 - val_accuracy: 0.7953
Epoch 7/10
215/215 [==============================] - 2s 8ms/step - loss: 0.4482 - accuracy: 0.8082 - val_loss: 0.4672 - val_accuracy: 0.7966
Epoch

In [175]:
pred6 = model_6.predict(val_sentences)

24/24 [==============================] - 0s 8ms/step


In [176]:
pred6 = [1 if i>0.5 else 0 for i in pred6 ]

In [177]:
calculate_results(val_target,pred6)

{'accuracy': 79.52755905511812,
 'precision': 0.7960691940776511,
 'recall': 0.7952755905511811,
 'f1': 0.7940558055501543}

In [179]:
model_7= tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(1,"sigmoid")
])

In [180]:
model_7.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])

In [181]:
model_7.fit(train_sentences,train_target,validation_data=(val_sentences,val_target),epochs=10)

Epoch 1/10
215/215 [==============================] - 4s 12ms/step - loss: 0.5066 - accuracy: 0.7803 - val_loss: 0.4468 - val_accuracy: 0.7979
Epoch 2/10
215/215 [==============================] - 2s 9ms/step - loss: 0.4147 - accuracy: 0.8130 - val_loss: 0.4356 - val_accuracy: 0.8097
Epoch 3/10
215/215 [==============================] - 2s 9ms/step - loss: 0.3999 - accuracy: 0.8218 - val_loss: 0.4316 - val_accuracy: 0.8150
Epoch 4/10
215/215 [==============================] - 2s 9ms/step - loss: 0.3926 - accuracy: 0.8264 - val_loss: 0.4274 - val_accuracy: 0.8150
Epoch 5/10
215/215 [==============================] - 2s 9ms/step - loss: 0.3864 - accuracy: 0.8285 - val_loss: 0.4292 - val_accuracy: 0.8189
Epoch 6/10
215/215 [==============================] - 2s 9ms/step - loss: 0.3795 - accuracy: 0.8332 - val_loss: 0.4245 - val_accuracy: 0.8189
Epoch 7/10
215/215 [==============================] - 2s 10ms/step - loss: 0.3737 - accuracy: 0.8340 - val_loss: 0.4240 - val_accuracy: 0.8202
Epoc